In [1]:
# Imports
import torch
from dask_jobqueue import SLURMCluster
from distributed import Client
from run_cocycles_mixed import run_experiment
from csuite_mixed import SCMS

In [2]:
seeds = 10
nsamples = 2000
corr = 0.0
use_dag = False
scms = list(SCMS.keys())

In [3]:
cluster = SLURMCluster(
    n_workers=0,
    memory="20GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10092",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
futures = []
for scm in scms:
    for seed in range(seeds):
        f = client.submit(run_experiment, 
                          sc_name = scm,
                          seed = seed,
                          use_dag = use_dag,
                          corr = corr,
                          N = nsamples)
        futures += [f] 

In [5]:
futures

[<Future: pending, key: run_experiment-24ae186e7cc5e55aac317083a8e4a9f4>,
 <Future: pending, key: run_experiment-bb44055ee286120e3ff87821ae5697be>,
 <Future: pending, key: run_experiment-609c6b0f0aebddcb85e079a8c00d7889>,
 <Future: pending, key: run_experiment-2a8a3d253b8c409a30a9bc95dc7506f3>,
 <Future: pending, key: run_experiment-208389d68eb85626cc45446ec6fa574f>,
 <Future: pending, key: run_experiment-38b7366786fe0b221a72b603ea68d25d>,
 <Future: pending, key: run_experiment-6ca5d55d125417be8a8a7965cf6edaaf>,
 <Future: pending, key: run_experiment-a7205d2569e85ef1fbf84f393694a239>,
 <Future: pending, key: run_experiment-b931d1c6eaa5fc86987c23eb77b11941>,
 <Future: pending, key: run_experiment-129fa3146a618f9b7cbd824612a99a7e>,
 <Future: pending, key: run_experiment-db9dea5a13b79556bde90c60c7c247c1>,
 <Future: pending, key: run_experiment-2119d40eef6ccbbcd6bdbc097ed17b51>,
 <Future: pending, key: run_experiment-93f1ea43c128b08ddbc3ccc2cbc40fac>,
 <Future: pending, key: run_experiment

In [ ]:
results = client.gather(futures)

In [ ]:
client.close()
cluster.close()

In [ ]:
torch.save(f = "cocycles_csuite_mixed_results_n={0}_corr={1}_dag={2}_trials={3}.pt".format(nsamples, corr, use_dag, trials), obj = results)

In [ ]:
results